# ***🧠NeuraQuiz: Neural-Powered Quiz Engine to generate, explain, and evaluate educational content.***

# Installing all the required Dependenices!!!!

In [ ]:
# Install dependencies
!pip install -q PyPDF2 google-generativeai openai
!pip install -q git+https://github.com/openai/whisper.git --no-deps
!pip install -q --upgrade google-generativeai
!pip install -q streamlit pyngrok
!pip install -q streamlit pyngrok ffmpeg-python

# Imports
import os
import google.generativeai as genai
import whisper
from PyPDF2 import PdfReader
from PIL import Image
import io
from IPython.display import display, Markdown


# Configure Gemini API with Kaggle Secrets:

In [ ]:
# Use Kaggle secrets for Gemini API
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
GOOGLE_API_KEY = user_secrets.get_secret("GOOGLE_API_KEY")  # Store in "Add a new secret"

genai.configure(api_key=GOOGLE_API_KEY)

# Generate MCQs from PDFs:

In [ ]:
def extract_text_from_pdf(file_path):
    reader = PdfReader(file_path)
    return " ".join([page.extract_text() for page in reader.pages])

def generate_mcqs_from_text(text):
    model = genai.GenerativeModel("models/gemini-1.5-pro-latest")
    prompt = (
        "You are an AI quiz generator. Based on the content below, generate exactly 5 MCQs.\n"
        "Return your output in this strict JSON format only:\n"
        "[\n"
        "  {\n"
        "    \"question\": \"...\",\n"
        "    \"options\": [\"A\", \"B\", \"C\", \"D\"],\n"
        "    \"answer\": \"B\",\n"
        "    \"explanation\": \"...\"\n"
        "  },\n"
        "  ...\n"
        "]\n"
        "Do NOT include any markdown, numbers, or additional comments.\n\n"
        f"Content:\n{text[:3000]}"
    )
    return model.generate_content(prompt).text


# ✅ Helper: Cleanly display MCQs (Kaggle Cell 6)
def display_mcqs(mcqs_raw):
    try:
        try:
            mcqs_data = json.loads(mcqs_raw)
        except json.JSONDecodeError:
            mcqs_data = eval(mcqs_raw, {"__builtins__": {}})

        if isinstance(mcqs_data, list):
            questions = mcqs_data
        elif isinstance(mcqs_data, dict) and "quiz" in mcqs_data:
            questions = mcqs_data["quiz"]
        else:
            display(Markdown("⚠️ **Unrecognized format:**"))
            display(Markdown(f"```\n{mcqs_raw}\n```"))
            return

        for i, q in enumerate(questions):
            display(Markdown(f"### Q{i+1}: {q.get('question', 'N/A')}"))
            for opt in q.get("options", []):
                display(Markdown(f"- {opt}"))
            display(Markdown(f"✅ **Answer:** {q.get('answer', 'N/A')}"))
            display(Markdown(f"💡 _Explanation_: {q.get('explanation', 'N/A')}"))
            display(Markdown("---"))

    except Exception as e:
        display(Markdown(f"❌ **Error parsing MCQs:** {e}"))
        display(Markdown(f"```\n{mcqs_raw}\n```"))


# Upload PDF
import ipywidgets as widgets
from IPython.display import display

# ✅ Upload widget and handler
uploader = widgets.FileUpload(accept='.pdf', multiple=False)
# display(Markdown("### 📄 Upload a PDF to generate MCQs"))
display(uploader)

def handle_pdf_upload(change):
    for name, file_info in uploader.value.items():
        with open(name, 'wb') as f:
            f.write(file_info['content'])
        text = extract_text_from_pdf(name)
        print("✅ Extracted text. Generating MCQs...")
        mcqs = generate_mcqs_from_text(text)
        display_mcqs(mcqs)

uploader.observe(handle_pdf_upload, names='value')


# Generate MCQs from an Image:

In [ ]:
def generate_mcqs_from_image(image):
    model = genai.GenerativeModel("models/gemini-1.5-pro-latest")
    prompt = (
        "You are an AI quiz generator. Based on this educational image, generate exactly 3 MCQs.\n"
        "Return your output in this strict JSON format only:\n"
        "[\n"
        "  {\n"
        "    \"question\": \"...\",\n"
        "    \"options\": [\"A\", \"B\", \"C\", \"D\"],\n"
        "    \"answer\": \"B\",\n"
        "    \"explanation\": \"...\"\n"
        "  },\n"
        "  ...\n"
        "]\n"
        "Do NOT include markdown or formatting — only return JSON."
    )
    return model.generate_content([prompt, image]).text


# -- MCQ DISPLAY FUNCTION --
def display_mcqs(mcqs_raw):
    try:
        try:
            mcqs_data = json.loads(mcqs_raw)
        except json.JSONDecodeError:
            mcqs_data = eval(mcqs_raw, {"__builtins__": {}})

        if isinstance(mcqs_data, list):
            questions = mcqs_data
        elif isinstance(mcqs_data, dict) and "quiz" in mcqs_data:
            questions = mcqs_data["quiz"]
        else:
            display(Markdown("⚠️ **Unrecognized format. Showing raw output:**"))
            display(Markdown(f"```\n{mcqs_raw}\n```"))
            return

        for i, q in enumerate(questions):
            display(Markdown(f"### Q{i+1}: {q.get('question', 'N/A')}"))
            for opt in q.get("options", []):
                display(Markdown(f"- {opt}"))
            display(Markdown(f"✅ **Answer:** {q.get('answer', 'N/A')}"))
            display(Markdown(f"💡 _Explanation_: {q.get('explanation', 'N/A')}"))
            display(Markdown("---"))

    except Exception as e:
        display(Markdown(f"❌ **Error parsing MCQs:** {e}"))
        display(Markdown(f"```\n{mcqs_raw}\n```"))

# -- IMAGE UPLOAD WIDGET --
img_uploader = widgets.FileUpload(accept='image/*', multiple=False)
display(Markdown("### 🖼️ Upload an image to generate quiz questions"))
display(img_uploader)

def handle_img_upload(change):
    for name, file_info in img_uploader.value.items():
        if file_info["metadata"]["type"].startswith("image/"):
            display(Markdown(f"📄 **Processing**: {name}"))
            mcqs = generate_mcqs_from_image(file_info['content'])
            display_mcqs(mcqs)
        else:
            display(Markdown("❌ Uploaded file is not a supported image."))

img_uploader.observe(handle_img_upload, names='value')


# Transcribe and Summarize Video Lectures:

In [ ]:
whisper_model = whisper.load_model("base")

def transcribe_and_summarize(file_path):
    transcript = whisper_model.transcribe(file_path)["text"]
    print("📜 Transcript extracted. Generating summary...")

    model = genai.GenerativeModel("models/gemini-1.5-pro-latest")
    response = model.generate_content(f"Summarize the following lecture in bullet points:\n{transcript[:3000]}")
    return response.text

video_uploader = widgets.FileUpload(accept='.mp4', multiple=False)
display(video_uploader)

def handle_video_upload(change):
    for name, file_info in video_uploader.value.items():
        with open(name, 'wb') as f:
            f.write(file_info['content'])
        summary = transcribe_and_summarize(name)
        display(Markdown(f"### 🎥 Lecture Summary:\n{summary}"))

video_uploader.observe(handle_video_upload, names='value')


# Evaluate Student Answers with Gemini:

In [ ]:
from IPython.display import display, Markdown

def evaluate_answer(student_ans, ideal_q, ideal_ans):
    prompt = f"""
    Evaluate the student's answer:
    Question: {ideal_q}
    Ideal Answer: {ideal_ans}
    Student's Answer: {student_ans}
    Give score out of 10 and feedback.
    """
    model = genai.GenerativeModel("models/gemini-1.5-pro-latest")
    response = model.generate_content(prompt)
    return response.text

# Example usage
student = "Photosynthesis happens in mitochondria."
ideal_q = "Where does photosynthesis occur in plant cells?"
ideal_ans = "In the chloroplasts."

feedback = evaluate_answer(student, ideal_q, ideal_ans)
display(Markdown(f"### 🧪 Evaluation:\n{feedback}"))


# Streamlit UI Code:

In [ ]:
streamlit_code = '''
import streamlit as st
import google.generativeai as genai
import whisper
from PyPDF2 import PdfReader
from PIL import Image
import tempfile
import os
import json
from kaggle_secrets import UserSecretsClient

# Load API Key
user_secrets = UserSecretsClient()
GOOGLE_API_KEY = user_secrets.get_secret("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

# Load Whisper model
whisper_model = whisper.load_model("base")

# Extract text from PDF
def extract_text_from_pdf(file):
    reader = PdfReader(file)
    return " ".join([page.extract_text() for page in reader.pages])

# Generate MCQs from PDF text
def generate_mcqs_from_text(text):
    model = genai.GenerativeModel("models/gemini-1.5-pro-latest")
    prompt = (
        "Generate 5 MCQs in JSON format like:\\n"
        "[{\\"question\\": \\"...\\", \\"options\\": [...], \\"answer\\": \\"...\\", \\"explanation\\": \\"...\\"}]\\n"
        f"Text:\\n{text[:3000]}"
    )
    return model.generate_content(prompt).text

# Generate MCQs from image
def generate_mcqs_from_image(image):
    model = genai.GenerativeModel("models/gemini-1.5-pro-latest")
    prompt = (
        "Analyze this educational image and generate 3 MCQs in strict JSON format like:\\n"
        "[{\\"question\\": \\"...\\", \\"options\\": [...], \\"answer\\": \\"...\\", \\"explanation\\": \\"...\\"}]\\n"
        "Do not use markdown, numbering, or commentary. Only return valid JSON."
    )
    return model.generate_content([prompt, image]).text

# Transcribe and summarize video
def transcribe_and_summarize(video_path):
    result = whisper_model.transcribe(video_path)
    transcript = result['text']
    model = genai.GenerativeModel("models/gemini-1.5-pro-latest")
    prompt = f"Summarize the following lecture in bullet points:\\n{transcript[:3000]}"
    return model.generate_content(prompt).text

# Evaluate student answer
def evaluate_student_answer(question, ideal, student):
    model = genai.GenerativeModel("models/gemini-1.5-pro-latest")
    prompt = f"Q: {question}\\nIdeal: {ideal}\\nStudent: {student}\\nGive score out of 10 and feedback."
    return model.generate_content(prompt).text

# Recommend learning path
def recommend_learning_path(level, goal):
    model = genai.GenerativeModel("models/gemini-1.5-pro-latest")
    prompt = f"I am a {level} student. My goal is to {goal}. Suggest a 2-week learning path."
    return model.generate_content(prompt).text
    
def render_quiz_ui(quiz_raw):
    import re

    st.markdown("## 📘 NeuraQuiz Results")
    try:
        # Step 1: Remove markdown fences like ```json
        cleaned = re.sub(r"```[a-zA-Z]*", "", quiz_raw).strip(\"` \\n\")

        # Step 2: Try parsing JSON
        try:
            quiz_data = json.loads(cleaned)
        except json.JSONDecodeError:
            # Fallback to eval
            quiz_data = eval(cleaned, {"__builtins__": {}})

        # Step 3: Extract questions
        if isinstance(quiz_data, dict) and "quiz" in quiz_data:
            questions = quiz_data["quiz"]
        elif isinstance(quiz_data, list):
            questions = quiz_data
        else:
            st.error("⚠️ Output is not a valid quiz format.")
            st.code(cleaned)
            return

        # Step 4: Render quiz
        for i, q in enumerate(questions):
            st.markdown(f"### Q{i+1}: {q.get('question', 'N/A')}")
            st.markdown("**Options**")
            labels = ["A", "B", "C", "D"]
            for idx, opt in enumerate(q.get("options", [])):
                label = labels[idx] if idx < len(labels) else f"Option {idx+1}"
                st.markdown(f"**{label})** {opt}")
            st.markdown(f"✅ **Answer:** {q.get('answer', 'N/A')}")
            st.markdown(f"💡 *Explanation*: {q.get('explanation', 'N/A')}")
            st.markdown("---")

    except Exception as e:
        st.error(f"❌ Error parsing quiz: {e}")
        st.code(quiz_raw)



# --- Streamlit App UI ---
st.set_page_config(page_title="NeuraQuiz", layout="centered")
st.markdown("# 🧠 NeuraQuiz")
st.markdown("##### Neural-Powered Quiz Engine to generate, explain, and evaluate educational content.")

option = st.sidebar.radio("Select Feature", [
    "📄 PDF to Quiz", "🖼️ Image to Quiz", "🎥 Video Summary", "🧪 Answer Evaluation", "🎯 Learning Path"
])

if option == "📄 PDF to Quiz":
    file = st.file_uploader("Upload a PDF", type=["pdf"])
    if file and st.button("Generate Quiz"):
        text = extract_text_from_pdf(file)
        quiz = generate_mcqs_from_text(text)
        render_quiz_ui(quiz)

elif option == "🖼️ Image to Quiz":
    file = st.file_uploader("Upload an image", type=["jpg", "jpeg", "png"])
    if file and st.button("Generate Quiz"):
        img = Image.open(file)
        quiz = generate_mcqs_from_image(img)
        render_quiz_ui(quiz)

elif option == "🎥 Video Summary":
    file = st.file_uploader("Upload a video (mp4)", type=["mp4"])
    if file and st.button("Transcribe & Summarize"):
        with tempfile.NamedTemporaryFile(delete=False, suffix=".mp4") as tmp:
            tmp.write(file.read())
            summary = transcribe_and_summarize(tmp.name)
        st.markdown(summary)

elif option == "🧪 Answer Evaluation":
    q = st.text_area("Question")
    ideal = st.text_area("Ideal Answer")
    student = st.text_area("Student Answer")
    if st.button("Evaluate"):
        feedback = evaluate_student_answer(q, ideal, student)
        st.markdown(f"### 🧪 Feedback:\\n{feedback}")

elif option == "🎯 Learning Path":
    level = st.selectbox("Student Level", ["Beginner", "Intermediate", "Advanced"])
    goal = st.text_input("Goal (e.g., learn calculus)")
    if st.button("Get Recommendation"):
        path = recommend_learning_path(level, goal)
        st.markdown(path)
'''
with open("streamlit_app.py", "w") as f:
    f.write(streamlit_code)


# Running NeuraQuiz Streamlit App in Kaggle by using ngrok:

In [ ]:
!ngrok config add-authtoken <YOUR TOKEN>

In [ ]:
import subprocess
from pyngrok import ngrok

port = 6006
public_url = ngrok.connect(port)
print("🔗 Your Streamlit App URL:", public_url)

# Start Streamlit with access to secrets
process = subprocess.Popen(["streamlit", "run", "streamlit_app.py", "--server.port", str(port)])

